In [1]:
import json
import requests
import os 
import re
from pprint import pprint
from dotenv import load_dotenv, dotenv_values
load_dotenv()

1a35c6cd110ec86b8d550c96ce7d7be1973db0be


In [2]:
from eyecite import get_citations, clean_text, resolve_citations, annotate_citations
from eyecite.models import FullCaseCitation, Resource
from eyecite.resolve import resolve_full_citation
from eyecite.tokenizers import HyperscanTokenizer

In [33]:
import eyecite

In [3]:
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import re

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/curtiswadsworth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/curtiswadsworth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def citation_parser(citation):

    pattern = re.compile(r"(?P<VOLUME>\d+)\s+(?P<REPORTER>[A-Za-z.\s]+)\s+(?P<PAGE>\d+)")
    match = pattern.match(citation)

    if match:
        volume = match.group("VOLUME")
        reporter = match.group("REPORTER")
        page = match.group("PAGE")
        
        #print(f"Volume: {volume}")
        #print(f"Reporter: {reporter}")
        #print(f"Page: {page}")
        return volume, reporter, page
    
    else:
        print("Invalid citation format")
        return None

In [ ]:
def get_cluster_by_citation(volume, reporter, page, token='your_api_token'):
    base_url = "https://www.courtlistener.com/api/rest/v3/clusters/"
    
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/json',
    }

    params = {
        'citations__volume': volume,
        'citations__reporter': reporter,
        'citations__page': page,
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        api_data = response.json()
        results = api_data.get('results', [])
        print(f"Cluster results: {results}")

        if results:
            # Return the first cluster found
            opinion_id = results[0]['id']
            print(f"Opinion Id: {opinion_id}")
            return opinion_id
    
    print(f"No cluster found for the specified citation.")
    return None

In [ ]:
def get_opinion_text(cite, search_by='id', token='your_api_token'):
    base_url = "https://www.courtlistener.com"

    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/json',
    }

    params = {
        search_by: cite
    }

    #search_url = "/api/rest/v3/opinions/"
    search_url = "/api/rest/v3/clusters/"
    response = requests.get(base_url + search_url, params=params, headers=headers)
    
    print(f"API URL: {response.url}")  # Print the API URL for debugging
    print(f"Request Headers: {response.request.headers}")  # Print the headers being sent
    print(f"API Status Code: {response.status_code}")  # Print the API status code
    #print(f"API Response Content: {response.text}")

    if response.status_code == 200:
        api_data = response.json()
        print(f"Parsed API Data: {api_data}")
        results = api_data.get('results', [])

        if results:
            first_result = results[0]
            plain_text = first_result.get('plain_text', '')
            html_content = first_result.get('html', '')
            if not html_content:  # If 'html' is empty, try 'html_lawbox'
                html_content = first_result.get('html_lawbox', '')

            return plain_text, html_content

        else: 
            print(f"No results text")
            return None

    print(f"No results found for {search_by}: {citation}")
    return None

In [ ]:
## TEST

def get_opinion_text(cite, search_by='docket_id', token='your_api_token'):
    base_url = "https://www.courtlistener.com"

    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/json',
    }

    params = {
        search_by: cite
    }
    
    search_url = "/api/rest/v3/dockets/"
    #search_url = "/api/rest/v3/opinions/" #retrieves opinion for 641 F. Supp. 828 (Poloroid)
    #search_url = "/api/rest/v3/clusters/"
    response = requests.get(base_url + search_url, params=params, headers=headers)
    
    print(f"API URL: {response.url}")  # Print the API URL for debugging
    print(f"Request Headers: {response.request.headers}")  # Print the headers being sent
    print(f"API Status Code: {response.status_code}")  # Print the API status code
    print(f"API Response Content: {response.text}")

    if response.status_code == 200:
        api_data = response.json()
        print(f"Parsed API Data: {api_data}")
        results = api_data.get('results', [])

        if results:
            first_result = results[0]
            plain_text = first_result.get('plain_text', '')
            html_content = first_result.get('html', '')

            if not html_content:
                html_content = first_result.get('html_lawbox', '')

            if not html_content:
                html_content = first_result.get('html_columbia', '')

            if not html_content:
                html_content = first_result.get('html_anon_2020', '')

            if not html_content:
                html_content = first_result.get('xml_harvard', '')

            if not html_content:
                html_content = first_result.get('html_with_citations', '')

            return plain_text, html_content

        else:
            print(f"No results text")
            return None
    
    print(f"No results found for {search_by}: {citation}")
    return None

In [ ]:
docket_id = '63778697'
search_field = 'id'
api_token = os.getenv("COURTLISTENER_API_KEY")

result = get_opinion_text(cite, search_by=search_field, token=api_token)
print(result)

In [ ]:
citation = "641 F. Supp. 828"
api_token = os.getenv("COURTLISTENER_API_KEY")

opinion_text = get_opinion_text_from_citation(citation, api_token)

In [ ]:
print(opinion_text)

In [ ]:
citation = "561 U.S. 593"
api_token = os.getenv("COURTLISTENER_API_KEY")

opinion_text = get_opinion_text_from_citation(citation, api_token)

In [ ]:
cite = "641 F. Supp. 828"
your_api_token = os.getenv("COURTLISTENER_API_KEY")  # Replace with your CourtListener API token

parsed_citations = citation_parser(cite)

volume = parsed_citations[0]
reporter = parsed_citations[1]
page = parsed_citations[2]

citation_id = get_cluster_by_citation(volume, reporter, page, token=your_api_token)

opinion_text = get_opinion_text(citation_id, search_by='id', token=your_api_token)

print(f"Opinion HTML: {opinion_text}")

In [ ]:
url = "https://www.courtlistener.com/api/rest/v3/opinions/"
response = requests.options(url)

print(f"Status Code: {response.status_code}")
print("Headers:")
pprint(dict(response.headers))


In [ ]:
cite = "6797301"
search_type = 'id' 
#cite = "641 F. Supp. 828"
#search_type = 'citation'
#search_type = 'citations' 

your_api_token = os.getenv('COURTLISTENER_API_KEY')  # Replace with your CourtListener API token

opinion_text_by_name = get_opinion_text(cite, search_by=search_type, token=your_api_token)

pprint(opinion_text_by_name)

In [ ]:
from urllib.parse import quote

## Opinions By docket_id

In [ ]:
def citation_parser(citation):

    pattern = re.compile(r"(?P<VOLUME>\d+)\s+(?P<REPORTER>[A-Za-z.\s]+)\s+(?P<PAGE>\d+)")
    match = pattern.match(citation)

    if match:
        volume = match.group("VOLUME")
        reporter = match.group("REPORTER")
        page = match.group("PAGE")
        
        #print(f"Volume: {volume}")
        #print(f"Reporter: {reporter}")
        #print(f"Page: {page}")
        return volume, reporter, page
    
    else:
        print("Invalid citation format")
        return None

In [ ]:
def get_cluster_by_citation(volume, reporter, page, token='API_token'):
    base_url = "https://www.courtlistener.com/api/rest/v3/clusters/"
    
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/json',
    }

    params = {
        'citations__volume': volume,
        'citations__reporter': reporter,
        'citations__page': page,
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        api_data = response.json()
        results = api_data.get('results', [])
        #print(f"Cluster results: {results}")

        if results:
            # Return the first cluster found
            docket_id = results[0]['docket_id']
            print(f"Docket Id: {docket_id}")
            return docket_id
    
    print(f"No cluster found for the specified citation.")
    return None

In [ ]:
def get_opinion_text(docket_id, token='API_token'):
    
    base_url = "https://www.courtlistener.com"

    headers = {
        'Authorization': f'Token {API_token}',
        'Accept': 'application/json',
    }

    search_url = f"/api/rest/v3/opinions/?cluster__docket={docket_id}"

    response = requests.get(base_url + search_url, headers=headers)

    print(f"API URL: {response.url}")  # Print the API URL for debugging
    print(f"Request Headers: {response.request.headers}")  # Print the headers being sent
    print(f"API Status Code: {response.status_code}")  # Print the API status code
    #print(f"API Response Content: {response.text}")


    if response.status_code == 200:
        api_data = response.json()
        results = api_data.get('results', [])

        print(f"Number of Results: {api_data['count']}")

        result_texts = []
        
        if response.status_code == 200:
            api_data = response.json()
            results = api_data.get('results', [])

            for i, result in enumerate(results, 1):
                print(f"Result {i} - Absolute URL: {result['absolute_url']}")

                plain_text = result.get('plain_text', '')
                html_content = result.get('html', '')

                if not html_content:
                    html_content = result.get('html_lawbox', '')

                if not html_content:
                    html_content = result.get('html_columbia', '')

                if not html_content:
                    html_content = result.get('html_anon_2020', '')

                if not html_content:
                    html_content = result.get('xml_harvard', '')

                if not html_content:
                    html_content = result.get('html_with_citations', '')
                
                result_texts.append({'plain_text': plain_text, 'html_content': html_content})

            return result_texts

    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
def get_opinion_text_from_citation(citation, token='API_token'):
    
    parsed_citations = citation_parser(citation)

    volume = parsed_citations[0]
    reporter = parsed_citations[1]
    page = parsed_citations[2]

    docket_id = get_cluster_by_citation(volume, reporter, page, token=token)

    opinion_text = get_opinion_text(docket_id, token=token)

    #print(f"Opinion HTML: {opinion_text}")
    return opinion_text

In [ ]:
citation = '641 F. Supp. 828'
API_token = os.getenv("COURTLISTENER_API_KEY")

result = get_opinion_text_from_citation(citation, token=API_token)

pprint(result)

In [ ]:
citation = '561 U.S. 593'
API_token = os.getenv("COURTLISTENER_API_KEY")

result = get_opinion_text_from_citation(citation, token=API_token)

pprint(result)

In [ ]:
docket_id = '967127'
API_token = os.getenv("COURTLISTENER_API_KEY")

result = get_opinion_text(docket_id, token=API_token)

pprint(result)

## Cleaning and Preprocessing

In [ ]:
def extract_references(html_text):
    
    statutes = re.findall(r'\b\d+\sU\.S\.C\.\s§\s\d+\b', html_text, re.IGNORECASE)
    
    citations = re.findall(r'\b(\d+\s[A-Za-z\s\.\d+]+(?:\d+)?(?:,)?(?:\s)?(?:at\s\d+)?(?:-\d+)?)', html_text, re.IGNORECASE)
    
    mpep = re.findall(r'\bMPEP(?:\s)?(?:§\s)?(?:chapter\s)?(?:\d+)(?:\.\d+)?\b', html_text, re.IGNORECASE)
    
    print(citations)
    
    return statutes, citations, mpep

In [ ]:
def clean_references(html_text, statutes, citations, mpep):
    soup = BeautifulSoup(html_text, 'html.parser')
    bs_text = soup.get_text(separator=' ')
    
    cleaned_text = re.sub(r'(ante|supra|infra|id)(?:,)?(\s+)(at\s\d+)(?:-\d+)?', 'CITATION_PLACEHOLDER', bs_text, flags=re.IGNORECASE)

    # Replace identified statutes, citations, and MPEP references
    cleaned_references = {
        'statutes': {},
        'citations': {},
        'mpep': {}
    }

    for statute in statutes:
        cleaned_text = cleaned_text.replace(statute, 'STATUTE_PLACEHOLDER ')
        cleaned_references['statutes'][statute] = 'STATUTE_PLACEHOLDER'

    for citation in citations:
        cleaned_text = cleaned_text.replace(citation, 'CITATION_PLACEHOLDER ')
        cleaned_references['citations'][citation] = 'CITATION_PLACEHOLDER'

    for mpep_ref in mpep:
        cleaned_text = cleaned_text.replace(mpep_ref, 'MPEP_PLACEHOLDER ')
        cleaned_references['mpep'][mpep_ref] = 'MPEP_PLACEHOLDER'

    #print(cleaned_text)
    return cleaned_text, cleaned_references

In [ ]:
def clean_and_tokenize(cleaned_text):
    # Remove HTML tags
    print(f"Incoming cleaned text: {cleaned_text}")
    soup = BeautifulSoup(cleaned_text, 'html.parser')
    processed_text = soup.get_text(separator=' ')
    
    print(processed_text)
    
    # Tokenize using WordPunctTokenizer (words and subwords)
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(processed_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]

    return tokens

In [ ]:
def remove_citations_clean_and_tokenize(html_text):
    
     # Remove line breaks
    html_text_no_space = html_text.replace('\n', ' ').replace('\r', '').replace("'", '')
    # Remove extra spaces
    cleaned_strings = re.sub(' +', ' ', html_text_no_space)
    
    if cleaned_strings == previous_text:
        return cleaned_strings
    
    print(f"Cleaned strings: {cleaned_strings}")
    
    statutes, citations, mpep = extract_references(cleaned_strings)
    
    cleaned_text, cleaned_references = clean_references(cleaned_strings, statutes, citations, mpep)
    
    tokens = clean_and_tokenize(cleaned_text)
    
    return tokens

In [ ]:
html_text = "<p>This is an example HTML text with 35 U.S.C. § 101, 383 U.S. 1, 10-11, 86 S. Ct. 684, 15 L. Ed. 2d 545 (1966), and MPEP § 602.01.</p>"

tokens = remove_citations_clean_and_tokenize(html_text)
print(tokens)

In [ ]:
html_text = "<p>This is an example HTML text with 35 U.S.C. § 101, 383 U.S. 1, 10-11, 86 S. Ct. 684, 15 L. Ed. 2d 545 (1966), and MPEP § 602.01.</p>"
cleaned_text, cleaned_references = clean_references(html_text)
print(cleaned_text)
print(cleaned_references)

In [ ]:
text = "<p>This is an example HTML text with 35 U.S.C. § 101, 383 U.S. 1, 10-11, 86 S. Ct. 684, 15 L. Ed. 2d 545 (1966), and MPEP § 602.01.</p>"

statutes, citations, mpep = extract_references(text)

print("Statutes:", statutes)
print("Citations:", citations)
print("MPEP:", mpep)

In [ ]:
html_text = """
'<opinion type="concurrence">\n'
                  '<author id="b881-11">Justice Breyer,</author>\n'
                  '<p id="ALrX">with whom Justice Scalia joins as to Part II, '
                  'concurring in the judgment.</p>\n'
                  '<p id="b881-12">I</p>\n'
                  '<p id="b881-13">I agree with Justice Stevens that a '
                  '“general method of engaging in business transactions” is '
                  'not a patentable “process” within the meaning of 35 U.S.C. '
                  '§ 101. <em>Ante, </em>at 614, 177 L. Ed. 2d, at 809 '
                  '(opinion concurring in judgment). This Court has never '
                  'be<page-number citation-index="1" '
                  'label="836">*836</page-number>fore held that so-called '
                  '“business methods” are patentable, and, in my view, the '
                  'text, history, and purposes of the Patent Act make clear '
                  'that they are not. <em>Ante, </em>at 621-657, 177 L. Ed. '
                  '2d, at 813-835 (same). I would therefore decide this case '
                  'on that ground, and I join Justice Stevens’ opinion in '
                  'full.</p>\n'
                  '<p id="b882-4">I write separately, however, in order to '
                  'highlight the substantial <em>agreement </em>among many '
                  'Members of the Court on many of the fundamental issues of '
                  'patent law raised by this case. In light of the need for '
                  'clarity and settled law in this highly technical area, I '
                  'think it appropriate to do so.</p>\n'
                  '<p id="b882-5">[561 U.S. 658]</p>\n'
                  '<p id="b882-6">II</p>\n'
                  '<p id="b882-7">In addition to the Court’s unanimous '
                  'agreement that the claims at issue here are unpatentable '
                  'abstract ideas, it is my view that the following four '
                  'points are consistent with both the opinion of the Court '
                  'and Justice Stevens’ opinion concurring in the '
                  'judgment:</p>\n'
                  '<p id="b882-8"><em>First, </em>although the text of § 101 '
                  'is broad, it is not without limit. See <em>ante, </em>at '
                  '601-602, 177 L. Ed. 2d, at 800-801 (opinion of the Court); '
                  '<em>ante, </em>at 622, 177 L. Ed. 2d, at 813-814 (opinion '
                  'of Stevens, J.). “[T]he underlying policy of the patent '
                  'system [is] that ‘the things which are worth to the public '
                  'the embarrassment of an exclusive patent,’ . . . must '
                  'outweigh the restrictive effect of the limited patent '
                  'monopoly.” <em>Graham </em>v. <em>John Deere Co. of Kansas '
                  'City, </em>383 U.S. 1, 10-11, 86 S. Ct. 684, 15 L. Ed. 2d '
                  '545 (1966) (quoting Letter from Thomas Jefferson to Isaac '
                  'McPherson (Aug. 13, 1813), in 6 Writings of Thomas '
                  'Jefferson 181 (H. Washington ed.)). The Court has thus been '
                  'careful in interpreting the Patent Act to “determine not '
                  'only what is protected, but also what is free for all to '
                  'use.” <em>Bonito Boats, Inc. </em>v. <em>Thunder Craft '
                  'Boats, Inc., </em>489 U.S. 141, 151, 109 S. Ct. 971, 103 L. '
                  'Ed. 2d 118 (1989). In particular, the Court has long held '
                  'that “[p]henomena of nature, though just discovered, mental '
                  'processes, and abstract intellectual concepts are not '
                  'patentable” under § 101, since allowing individuals to '
                  'patent these fundamental principles would “wholly preempt” '
                  'the public’s access to the “basic tools of scientific and '
                  'technological work.” <em>Gottschalk </em>v. <em>Benson, '
                  '</em>409 U.S. 63, 67, 72, 93 S. Ct. 253, 34 L. Ed. 2d 273 '
                  '(1972); see also, <em>e.g., Diamond </em>v. <em>Diehr, '
                  '</em>450 U.S. 175, 185, 101 S. Ct. 1048, 67 L. Ed. 2d 155 '
                  '(1981); <em>Diamond </em>v. <em>Chakrabarty, </em>447 U.S. '
                  '303, 309, 100 S. Ct. 2204, 65 L. Ed. 2d 144 (1980).</p>\n'
                  '<p id="b882-11"><em>Second, </em>in a series of cases that '
                  'extend back over a century, the Court has stated that '
                  '“[t]ransformation and reduction of an article to a '
                  'different state or thing is <em>the clue </em>to the '
                  'patent-ability of a process claim that does not include '
                  'particular machines.” <em>Diehr, supra, </em>at 184, 101 S. '
                  'Ct. 1048, 67 L. Ed. 2d 155 (emphasis added; internal '
                  'quotation marks omitted); see also, <em>e.g., Benson, '
                  'supra, </em>at 70, 93 S. Ct. 253, 34 L. Ed. 2d 273; '
                  '<em>Parker </em>v. <em>Flook, </em>437 U.S. 584, 588, n. 9, '
                  '98 S. Ct. 2522, 57 L. Ed. 2d 451 (1978); <em>Cochrane '
                  '</em>v. <em>Deener, </em>94 U.S. 780, 788, 24 L. Ed. 139 '
                  '(1877). Application of</p>\n'
                  '<p id="b882-12">[561 U.S. 659]</p>\n'
                  '<p id="b882-13">this test, the so-called '
                  '“machine-or-transformation test,” has thus repeatedly '
                  'helped the Court to determine what is “a patentable '
                  '‘process.’ ” <em>Flook, supra, </em>at 589, 98 S. Ct. 2522, '
                  '57 L. Ed. 2d 451.</p>\n'
                  '<p id="b882-15"><em>Third, </em>while the '
                  'machine-or-transformation test has always been a “useful '
                  'and important clue,” it has never been the “sole test” for '
                  'deter<page-number citation-index="1" '
                  'label="837">*837</page-number>mining patentability. '
                  '<em>Ante, </em>at 604, 177 L. Ed. 2d, at 803; see also '
                  '<em>ante, </em>at 614, 177 L. Ed. 2d, at 808 (opinion of '
                  'Stevens, J.); <em>Benson, supra, </em>at 71, 93 S. Ct. 253, '
                  '34 L. Ed. 2d 273 (rejecting the argument that “no process '
                  'patent could ever qualify” for protection under § 101 “if '
                  'it did not meet the [machine-or-transformation] '
                  'requirements”). 
                    """
tokens = remove_citations_clean_and_tokenize(html_text)
print(tokens)


In [ ]:
def extract_references_test(html_text):
    
    statutes = re.findall(r'\b\d+\sU\.S\.C\.\s§\s\d+\b', html_text, re.IGNORECASE)
    
    citations = re.findall(r'\b(\d+\s[A-Za-z\s\.\d+]+(?:\d+)?(?:,)?(?:\s)?(?:at\s\d+)?(?:-\d+)?)', html_text, re.IGNORECASE)
    
    mpep = re.findall(r'\bMPEP(?:\s)?(?:§\s)?(?:chapter\s)?(?:\d+)(?:\.\d+)?\b', html_text, re.IGNORECASE)
    
    return cleaned_text, statutes, citations, mpep

In [ ]:
def extract_and_replace_citations(text):
    '''
    Input text including legal citations.
    Clean text using eyecite.clean_text.
    
    '''
    # Use eyecite to remove white space and clean text using eyecite.clean_text.
    clean_text = eyecite.clean_text(text, ['html', 'all_whitespace'])
    
    # Use eyecite to find legal citations in the text
    citations = eyecite.get_citations(clean_text)

    return citations

In [81]:
def extract_and_replace_citations(text):
    '''
    Input text including legal citations.
    Clean text using eyecite.clean_text.
    
    '''
    # Use eyecite to remove white space and clean text using eyecite.clean_text.
    cleaned_text = clean_text(text, ['html', 'all_whitespace'])
    
    # Use eyecite to find legal citations in the text
    citations = get_citations(cleaned_text)
    
    # Placeholder for each type of citation
    placeholders = {
        'FullLawCitation': 'FULL_LAW_CITATION_PLACEHOLDER',
        'FullCaseCitation': 'FULL_CASE_CITATION_PLACEHOLDER',
        'ShortCaseCitation': 'SHORT_CASE_CITATION_PLACEHOLDER',
        'IdCitation': 'ID_CITATION',
        'SupraCitation': 'SUPRA_CITATION',
        'UnknownCitation': 'UNKNOWN_CITATION_PLACEHOLDER',
    }

    output_text = cleaned_text
    
    # Replace each citation with the corresponding placeholder
    for citation in citations:
        citation_type = type(citation).__name__
        print(citation_type)
        
        placeholder = placeholders.get(citation_type, '')

        if citation_type == 'FullCaseCitation':
            output_text = output_text.replace(citation.matched_text(), placeholder)
            print(citation.matched_text())
        elif citation_type == 'FullLawCitation':
            output_text = output_text.replace(citation.matched_text(), placeholder)
            print(citation.matched_text())
        elif citation_type == 'ShortCaseCitation':
            output_text = output_text.replace(citation.matched_text(), placeholder)
            print(citation.matched_text())
        #elif citation_type == 'IdCitation':
        #    output_text = output_text.replace(citation.matched_text(), placeholder)
        #    print(citation.matched_text())
        #elif citation_type == 'SupraCitation':
        #    output_text = output_text.replace(citation.matched_text(), placeholder)
        #    print(citation.matched_text())
        #elif citation_type == 'UnknownCitation':
        #    output_text = output_text.replace(citation.matched_text(), placeholder)
        #    print(citation.matched_text())
        else:
            pass
            
            
    return citations, output_text, cleaned_text

In [83]:
citations, text_with_placeholders, cleaned_text = extract_and_replace_citations(sample_text)

pprint(citations)
#print(f'Extracted {len(citations)} citations.\n')
#print(f'First citation:\n {citations[4]}\n')

#citations[4].token

print(cleaned_text)
print(text_with_placeholders)

FullLawCitation
Public Law 112-29, 125
UnknownCitation
IdCitation
Id.
SupraCitation
supra,
supra,
ShortCaseCitation
177 L. Ed. 2d, at 809
UnknownCitation
FullCaseCitation
489 U.S. 141
FullCaseCitation
109 S. Ct. 971
FullCaseCitation
103 L. Ed. 2d 118
[FullLawCitation('Public Law 112-29, 125', groups={'reporter': 'Public Law', 'title': '112-29', 'section': '125'}, metadata=FullLawCitation.Metadata(parenthetical=None, pin_cite=None, year=None, publisher=None, day=None, month=None)),
 UnknownCitation('§', metadata=CitationBase.Metadata(parenthetical=None)),
 IdCitation('Id.', metadata=IdCitation.Metadata(parenthetical=None, pin_cite='at 410')),
 SupraCitation('supra,', metadata=SupraCitation.Metadata(parenthetical=None, antecedent_guess='765', pin_cite='1909', volume=None)),
 ShortCaseCitation('177 L. Ed. 2d, at 809', groups={'volume': '177', 'reporter': 'L. Ed. 2d', 'page': '809'}, metadata=ShortCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court=None, antecedent_gu

In [35]:
print(dir(eyecite))

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__pdoc__', '__spec__', 'annotate', 'annotate_citations', 'clean', 'clean_text', 'find', 'get_citations', 'helpers', 'models', 'regexes', 'resolve', 'resolve_citations', 'tokenizers', 'utils']


In [36]:
print(dir(eyecite.helpers))

['BACKWARD_SEEK', 'CaseCitation', 'CitationBase', 'CitationToken', 'FullCaseCitation', 'FullJournalCitation', 'FullLawCitation', 'List', 'MAX_MATCH_CHARS', 'Optional', 'POST_FULL_CITATION_REGEX', 'POST_JOURNAL_CITATION_REGEX', 'POST_LAW_CITATION_REGEX', 'POST_SHORT_CITATION_REGEX', 'ParagraphToken', 'ResourceCitation', 'StopWordToken', 'Token', 'Tokens', 'Tuple', 'YEAR_REGEX', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_highest_valid_year', 'add_defendant', 'add_journal_metadata', 'add_law_metadata', 'add_post_citation', 'cast', 'clean_pin_cite', 'courts', 'date', 'disambiguate_reporters', 'extract_pin_cite', 'get_court_by_paren', 'get_year', 'joke_cite', 'match_on_tokens', 'process_parenthetical', 're', 'strip_punct']


In [75]:
sample_text = '''Art. 1, Sec. 8. Leahy-Smith America Invents Act (AIA), Public Law 112-29, 125 Stat. 284., 35 U.S.C. 1 Establishment.
ante,  at 510-980, infra at 732, mpep § 701.32, mpep chapter 2100, Id. at 410, id at 765, supra, 1909, id, 465
177 L. Ed. 2d, at 809
mpep § 701.32, mpep chapter 2100
Bonito Boats, Inc. v. Thunder Craft Boats, Inc., 489 U.S. 141, 151, 109 S. Ct. 971, 103 L. Ed. 2d 118 (1989)
eyecite
'''

In [13]:
def extract_and_replace_citations(text):
    '''
    Input text including legal citations.
    Clean text using eyecite.clean_text.
    
    '''
    # Use eyecite to remove white space and clean text using eyecite.clean_text.
    cleaned_text = clean_text(text, ['html', 'all_whitespace'])
    
    # Use eyecite to find legal citations in the text
    citations = get_citations(cleaned_text)

    # Placeholder for each type of citation
    placeholders = {
        'FullLawCitation': 'FULL_LAW_CITATION_PLACEHOLDER',
        'FullCaseCitation': 'FULL_CASE_CITATION_PLACEHOLDER',
        'ShortCaseCitation': 'SHORT_CASE_CITATION_PLACEHOLDER',
        'UnknownCitation': 'UNKNOWN_CITATION_PLACEHOLDER',
    }

    # Replace each citation with the corresponding placeholder
    for citation in citations:
        citation_type = type(citation).__name__
        placeholder = placeholders.get(citation_type, '')
        
        #print(citation.span)
        #print(citation.span_start)
        #print(citation.span_end)
        #print(f"Matched Text: {citation.matched_text}")
        #print(f"Full Span: {citation.__match_args__}")
        #for attr in dir(citation):
        #   print(attr)
        #parenthetical = citation.metadata.parenthetical
        #print(f"Parenthetical: {parenthetical}")

        start = citation.span_start
        end = citation.span_end

        ouput_text = cleaned_text[:start] + placeholder + cleaned_text[end:]

    return citations, text

In [15]:
#citation = <bound method CitationBase.matched_text of FullLawCitation('Public Law 112-29, 125', groups={'reporter': 'Public Law', 'title': '112-29', 'section': '125'}, metadata=FullLawCitation.Metadata(parenthetical=None, pin_cite=None, year=None, publisher=None, day=None, month=None))>


# Accessing the matched text
for citation in citations: 
    matched_text = citation.matched_text()
    print(matched_text)

# Accessing groups
groups = citation.groups

# Accessing specific group values
reporter = citation.groups['reporter']
title = citation.groups['title']
section = citation.groups['section']

# Accessing metadata
metadata = citation.metadata

# Accessing specific metadata values
parenthetical = citation.metadata.parenthetical
pin_cite = citation.metadata.pin_cite
year = citation.metadata.year
publisher = citation.metadata.publisher
day = citation.metadata.day
month = citation.metadata.month

TypeError: 'FullCaseCitation' object does not support item assignment

In [ ]:
citations = extract_and_replace_citations(sample_text)

for citation in citations:
    print(f"Type: {type(citation).__name__}")
    print(f"Attributes and methods: {dir(citation)}")

In [17]:
first_citation = None

for citation in citations:
    matched_text = citation.matched_text()
    #print(matched_text)

    # Store the matched text of the first citation
    if first_citation is None:
        first_citation = matched_text
        break

# Now 'first_citation' contains the matched text of the first citation
print("First Citation:", first_citation)

first_citation_type = type(first_citation)
print("Type of First Citation:", first_citation_type)

First Citation: Public Law 112-29, 125
Type of First Citation: <class 'str'>


In [88]:
def extract_legal_tokens(text):
    # Define regular expressions for different legal citation patterns
    statute_pattern = r'\d+\s+[A-Za-z]+\s+\d+'
    usc_pattern = r'\d+\s+U\.S\.C\.\s+\d+'
    manual_pattern = r'mpep\s+§\s+\d+\.\d+'
    chapter_pattern = r'mpep\s+chapter\s+\d+'
    short_citation_pattern = r'(ante|at|infra|Id\.|supra)\s+at?\s*\d+-\d+'

    # Combine patterns into a single pattern
    combined_pattern = f'({statute_pattern}|{usc_pattern}|{manual_pattern}|{chapter_pattern}|{short_citation_pattern})'

    # Find all matches in the text
    matches = re.findall(combined_pattern, text)
    
    # Extract labeled tokens from matches
    labeled_tokens = []
    for match in matches:
        labeled_token = {k: v for k, v in match.groupdict().items() if v is not None}
        labeled_tokens.append(labeled_token)

    return labeled_tokens

    return matches

In [89]:
legal_tokens = extract_legal_tokens(sample_text)
print(legal_tokens)

AttributeError: 'tuple' object has no attribute 'groupdict'

In [ ]:
sample_text = '''Art. 1, Sec. 8. Leahy-Smith America Invents Act (AIA), Public\nLaw 112-29, 125
Stat. 284. 35 U.S.C. 1 Establishment.

ante,  at 510-980, infra at 732, mpep § 701.32, mpep chapter 2100, Id. at 656, id. at 987, ID, 1009.

177 L. Ed. 2d, at 809

Ante, at 614, 177 L. Ed. 2d, at 809, 197 L. Ed. 2d, at 809-901, [561 U.S. 658], MPEP 2000
Bonito Boats, Inc. v. Thunder Craft Boats, Inc., 489 U.S. 141, 151, 109 S. Ct. 971, 103 L. Ed. 2d 118 (1989)
Gottschalk v. Benson, 409 U.S. 63, 67, 72, 93 S. Ct. 253, 34 L. Ed. 2d 273 (1972)

'''

In [ ]:
sample_text = '''"INTRODUCTION\n  Constitutional Basis\nThe Constitution of the United States
provides:\n“Art. 1, Sec. 8. The Congress shall have power . . .\nTo promote the progress
of science and useful arts,\nby securing for limited times to authors and inventors\nthe
exclusive right to their respective writings and\ndiscoveries.”\n  Statutes\nPursuant to
the provision of the Constitution,\nCongress has over the years passed a number of\nstatutes
under which the U.S. Patent and Trademark\nOffice (USPTO) is organized and our patent system\ni
s established. The provisions of the statutes can in\nno way be changed or waived by the USPTO.
\nPrior to January 1, 1953, the law relating to patents\nconsisted of various sections of the
Revised Statutes\nof 1874, derived from the Patent Act of 1870 and\nnumerous amendatory and 
additional acts.\nBy an Act of Congress approved July 19, 1952,\nwhich came into effect on 
January 1, 1953, the patent\nlaws were revised and codified in Title 35 of the\nUnited States Code.
In referring to a particular\nsection of the patent code the citation is given, for\nexample, as, 
35 U.S.C. 1. The United States Code\nis available online at http://uscode.house.gov/.\nUpon occasion, 
additional provisions pertaining to\npatents are set forth in a Public Law but are not\ncodified. 
Examples are some sections of the\nLeahy-Smith America Invents Act (AIA), Public\nLaw 112-29, 125
Stat. 284. The Public Laws are\navailable at www.congress.gov/public-laws.\nTitle 35 of the United
States Code and sections 14,\n18, and 33 of the AIA are reproduced in Appendix\nL of the Manual 
of Patent Examining Procedure\n(MPEP), however the Public Laws are the\nauthoritative source and
should be consulted if a\nneed arises to verify the authenticity of the language\nreproduced in the
MPEP. A copy of the consolidated\npatent laws that incorporates any statutory revisions\nthat became
effective subsequent to the latest\nrevision of the MPEP is available on the USPTO\nwebsite at www.
uspto.gov/web/offices/pac/mpep/\nconsolidated_laws.pdf.\n35 U.S.C. 1 Establishment.\n  (a) 
ESTABLISHMENT.— The United States Patent and\nTrademark Office is established as an agency of the 
United\nStates, within the Department of Commerce. In carrying out its\nfunctions, the United States
Patent and Trademark Office shall\nbe subject to the policy direction of the Secretary of Commerce,
\nbut otherwise shall retain responsibility for decisions regarding\nthe management and administration
of its operations and shall\nexercise independent control of its budget allocations and\nexpenditures,
personnel decisions and processes, procurements,\nand other administrative and management functions
in\naccordance with this title and applicable provisions of law. Those\noperations designed to grant
and issue patents and those\noperations which are designed to facilitate the registration 
of\ntrademarks shall be treated as separate operating units within\nthe Office.\n

ante,  at 510-980, infra at 732, mpep § 701.32, mpep chapter 2100

177 L. Ed. 2d, at 809

Ante, at 614, 177 L. Ed. 2d, at 809, 197 L. Ed. 2d, at 809-901, [561 U.S. 658], MPEP 2000
Bonito Boats, Inc. v. Thunder Craft Boats, Inc., 489 U.S. 141, 151, 109 S. Ct. 971, 103 L. Ed. 2d 118 (1989)
Gottschalk v. Benson, 409 U.S. 63, 67, 72, 93 S. Ct. 253, 34 L. Ed. 2d 273 (1972)

'''

clean_text = eyecite.clean_text(sample_text, ["all_whitespace"])

citations = eyecite.get_citations(clean_text)

text_with_placeholders = extract_and_replace_citations(sample_text)


print(citations)
print('-----')
#print(sample_text)
print(clean_text)
print('-----')
print(text_with_placeholders)

In [ ]:
pip install hyperscan

## MPEP PDF to text

In [ ]:
import fitz  # PyMuPDF

In [ ]:
def convert_pdfs_to_text(folder_path):
    """
    Convert PDF files in the specified folder to text.

    Parameters:
    - folder_path: The path to the folder containing PDF files.

    Returns:
    - A dictionary mapping filenames to their corresponding text content.
    """
    pdf_texts = {}

    # Check if the folder path exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return pdf_texts

    # Loop through PDF files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            pdf_document = None

            try:
                # Open the PDF file
                pdf_document = fitz.open(file_path)

                # Extract text from each page
                text = ""
                for page_number in range(pdf_document.page_count):
                    page = pdf_document[page_number]
                    text += page.get_text()

                # Save text content in the dictionary
                pdf_texts[filename] = text

            except Exception as e:
                print(f"Error processing '{filename}': {str(e)}")

            finally:
                # Close the PDF document
                if pdf_document is not None:
                    pdf_document.close()

    return pdf_texts

In [ ]:
def clean_and_tokenize_pdf(text):
    processed_texts = {}

    # Process each text in the original dictionary
    for filename, text in pdf_texts.items():
        # Assuming remove_citations_clean_and_tokenize is your function
        processed_text = remove_citations_clean_and_tokenize_1(text)
        #print(filename)

        # Store the processed text in the new dictionary
        processed_texts[filename] = processed_text
    
    return processed_texts

In [ ]:
def remove_citations_clean_and_tokenize_1(html_text, previous_text=None):
    # Remove line breaks
    html_text_no_space = html_text.replace('\n', ' ').replace('\r', '').replace("'", '')
    
    # Remove extra spaces
    cleaned_strings = re.sub(' +', ' ', html_text_no_space)
    
    #print(f"Cleaned strings: {cleaned_strings}")
    
    # Terminate recursion if no change in text
    if cleaned_strings == previous_text:
        return cleaned_strings
    
    statutes, citations, mpep = extract_references(cleaned_strings)
    
    cleaned_text, cleaned_references = clean_references(cleaned_strings, statutes, citations, mpep)
    
    tokens = clean_and_tokenize(cleaned_text)
    
    # Recursive call with updated text
    return tokens

In [ ]:
folder_path = "/Users/curtiswadsworth/NL_Notebooks/MPEP_pdf_1"
pdf_texts = convert_pdfs_to_text(folder_path)
print(pdf_texts)

processed_texts = clean_and_tokenize_pdf(pdf_texts)

print(processed_texts)

In [3]:
import re

pattern = re.compile(r"(id(?:\.)?,?|ibid(?:\.)?,?)")

# Test cases
test_strings = ["id.", "id,", "ibid.", "ibid,", "id", "ibid", "id.,"]

for test_string in test_strings:
    match = pattern.search(test_string)
    if match:
        print(f'Matched "{match.group()}" in "{test_string}"')
    else:
        print(f'No match in "{test_string}"')

Matched "id." in "id."
Matched "id," in "id,"
Matched "ibid." in "ibid."
Matched "ibid," in "ibid,"
Matched "id" in "id"
Matched "ibid" in "ibid"
Matched "id.," in "id.,"


In [14]:

MPEP_REGEX = re.compile(r'\bmpep\b|\bm\.p\.e\.p\b', flags=re.IGNORECASE)
matches = MPEP_REGEX.finditer("MPEP is an example, mpep is annother, and m.p.e.p example.")

for match in matches:
    print(f'Matched: {match.group()}')  # Access the capturing group)

Matched: MPEP
Matched: mpep
Matched: m.p.e.p
